In [2]:
!pip install llama-index

In [3]:
!pip install llama-index-vector-stores-chroma

  Using cached llama_index_core-0.11.23-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.11.23-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.33.post1
    Uninstalling llama-index-core-0.12.33.post1:
      Successfully uninstalled llama-index-core-0.12.33.post1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.12.22 requires llama-index-core<0.13.0,>=0.12.22, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-agent-openai 0.4.6 requires llama-index-core<0.13.0,>=0.12.18, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-cli 0.4.1 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-embeddings-huggingface 0.5.2 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-embeddings-openai 0.3.1 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-indices-managed-llama-cloud 0.6.8 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
lla

In [4]:
!pip install llama-index-embeddings-huggingface

  Using cached llama_index_core-0.12.33.post1-py3-none-any.whl.metadata (2.6 kB)
Using cached llama_index_core-0.12.33.post1-py3-none-any.whl (7.7 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.23
    Uninstalling llama-index-core-0.11.23:
      Successfully uninstalled llama-index-core-0.11.23


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-vector-stores-chroma 0.2.0 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.33.post1 which is incompatible.


In [ ]:

import pandas as pd
# Import LlamaIndex core classes for creating documents, indexing, and storage
from llama_index.core import VectorStoreIndex, Document, StorageContext
# Import ChromaVectorStore to use ChromaDB as the vector store
from llama_index.vector_stores.chroma import ChromaVectorStore
# Import the ChromaDB library to create and manage the vector database
import chromadb
# Import SentenceSplitter to split documents into smaller chunks for embedding
from llama_index.core.node_parser import SentenceSplitter
# Import HuggingFaceEmbedding to generate embeddings for the documents
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# Import os for handling file paths and directories
import os
# Step 1: Data Processing - Read the CSV file into a pandas DataFrame
df = pd.read_csv('incidents_materiels.csv', sep=';', encoding='utf-8')
documents = []


for index, row in df.iterrows():
    content = " ".join(str(value) for value in row.values)
    doc = Document(text=content, metadata={"row_id": index, "source": "incidents_materiels.csv"})
   # Details:
    # - 'text=content': The combined string from the row.
    # - 'metadata={"row_id": index, "source": "incidents_materiels.csv"}': Metadata for tracking:
    #   - 'row_id': The row number (e.g., 0) to trace back to the CSV.
    #   - 'source': The file name, for provenance.
    documents.append(doc)
# Step 3: Chunking - Initialize a SentenceSplitter for splitting documents into smaller chunks
node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
# Step 4: Embedding - Initialize the embedding model using HuggingFace's BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Step 5: ChromaDB Creation - Initialize a persistent ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection(name="csv_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Step 6: Storage Context - Create a StorageContext to manage storage operations
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# Step 7: Create the Vector Store Index - Build the index for querying
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model,
    transformations=[node_parser]
)



c:\Users\Amina\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [6]:
print("\nInspecting the first 5 entries in ChromaDB collection 'csv_collection':")
results = chroma_collection.get(limit=5, include=['documents', 'metadatas', 'embeddings'])  # Retrieve up to 5 entries, including embeddings
# What: Uses 'chroma_collection.get' to fetch entries, explicitly including embeddings.
# Why: 'get' retrieves stored data (text, metadata, embeddings, IDs) from ChromaDB.
# Details:
# - 'limit=5': Limits the number of entries to 5 for inspection.
# - 'include=['documents', 'metadatas', 'embeddings']':
for i, (doc_text, metadata, embedding) in enumerate(zip(results['documents'], results['metadatas'], results['embeddings'])):
    # What: Loops through the retrieved documents, metadata, and embeddings.
    # Why: Prints each entry in a readable format, including a preview of the embeddings.
    # Details: 'zip' pairs each text chunk, metadata, and embedding for iteration.
    print(f"\nEntry {i + 1}:")
    print(f"Text Chunk: {doc_text}")
    print(f"Metadata: {metadata}")
    print(f"Embedding Shape: {len(embedding)} dimensions")
    print(f"Embedding Preview (first 5 elements): {embedding[:5]}")


Inspecting the first 5 entries in ChromaDB collection 'csv_collection':

Entry 1:
Text Chunk: Clavier de portable qui surchauffe Le clavier devient anormalement chaud pendant l'utilisation. Nettoyer les ventilateurs et vérifier la circulation d'air. 2
Metadata: {'_node_content': '{"id_": "0093bcea-834f-4ab3-aba6-a602ccd7ae0e", "embedding": null, "metadata": {"row_id": 136, "source": "incidents_materiels.csv"}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {"1": {"node_id": "4a6b7fc5-64e2-4c5e-9034-83874ef3c750", "node_type": "4", "metadata": {"row_id": 136, "source": "incidents_materiels.csv"}, "hash": "c0c5210df75ea198c42fa2a65a0e58d343df1f8157ee08d9d8149f2e7bbb16d0", "class_name": "RelatedNodeInfo"}}, "metadata_template": "{key}: {value}", "metadata_separator": "\\n", "text": "", "mimetype": "text/plain", "start_char_idx": 0, "end_char_idx": 157, "metadata_seperator": "\\n", "text_template": "{metadata_str}\\n\\n{content}", "class_name": "Tex

In [7]:
# Step 9: Test the retrieval component of the RAG system without an LLM
print("\nTesting the retrieval component of the RAG system (without LLM synthesis):")
# Embed the query using the same embed_model used for the documents
query = "Quels sont les problèmes liés à l'accès Internet ?"
query_embedding = embed_model.get_text_embedding(query)
# Query ChromaDB directly, ensuring no 'where' clause is passed
results = chroma_collection.query(
    query_embeddings=[query_embedding],
    n_results=2,
    include=["documents", "metadatas", "distances"]
)
# 'query_embeddings': The embedding of the query.
# 'n_results=2': Retrieve the top 2 most similar chunks.
# No 'where' clause is passed, avoiding the ValueError.
print(f"Query: {query}")
print("Retrieved Chunks:")
for i, (doc_text, metadata, distance) in enumerate(zip(results["documents"][0], results["metadatas"][0], results["distances"][0])):
    # ChromaDB returns distances (1 - cosine similarity); convert to similarity score
    similarity_score = 1 - distance
    print(f"\nChunk {i + 1}:")
    print(f"Text: {doc_text}")
    print(f"Metadata: {metadata}")
    print(f"Similarity Score: {similarity_score}")


Testing the retrieval component of the RAG system (without LLM synthesis):
Query: Quels sont les problèmes liés à l'accès Internet ?
Retrieved Chunks:

Chunk 1:
Text: Interférences WiFi avec micro-ondes Le WiFi se coupe lorsque le micro-ondes est utilisé. Éloigner le routeur ou utiliser une bande 5 GHz. 0
Metadata: {'_node_content': '{"id_": "1a0f8585-52f8-4c88-bcd8-b69b7bce80eb", "embedding": null, "metadata": {"row_id": 106, "source": "incidents_materiels.csv"}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {"1": {"node_id": "65326b94-baed-4b20-bf83-6ed7d7b5dd77", "node_type": "4", "metadata": {"row_id": 106, "source": "incidents_materiels.csv"}, "hash": "728f40b345fe158044c84afaa8e8e7ed05a550d86e1877496e0daef5c33c60c2", "class_name": "RelatedNodeInfo"}}, "metadata_template": "{key}: {value}", "metadata_separator": "\\n", "text": "", "mimetype": "text/plain", "start_char_idx": 0, "end_char_idx": 139, "metadata_seperator": "\\n", "text_templat

In [8]:
!pip install llama-index-llms-huggingface-api

In [13]:
# Step 10: Import Libraries for the Chatbot Layer
from llama_index.llms.ollama import Ollama  # To use Mistral 7B via Ollama locally
from llama_index.core.chat_engine import ContextChatEngine  # For conversational memory and chat functionality

# Step 11: Set Up the LLM (Mixtral 8x7B via Ollama)
# Configure Mixtral 8x7B using Ollama running locally
llm = Ollama(
    model="mistral:latest",  # Specify the model name as used in Ollama
    base_url="http://localhost:11434",  # Default Ollama API endpoint (adjust if different)
    request_timeout=360.0,  # Increase timeout for large models like Mixtral
    temperature=0.7,  # Controls creativity; matches your original setting
    top_p=0.9,  # Controls diversity; matches your original setting
    max_tokens=512  # Limits response length; matches your original setting
)


# Step 12: Define the Custom Prompt for Intent Engineering
custom_prompt = (
    "You are a helpful assistant specializing in IT-related incidents and technical tasks in a technical environment, including drone programming, SDK usage, network issues, hardware problems, and software troubleshooting. "
    "Dynamically detect the language of the user’s query (English or French) and respond in the same language, sounding like a friendly, knowledgeable colleague. "
    "Intelligently interpret, interpret the query, even if it contains typos or ambiguity, to accurately understand the user’s intent (e.g., 'wifii prblem' as 'WiFi problem' or 'rpmpts' as 'prompts'). "
    "For queries related to IT incidents or tasks (e.g., drone control, data storage, network issues, hardware, software): "
    "- Retrieve and use only the provided context (e.g., database or knowledge base) to craft a concise, accurate response. "
    "- If the context contains a direct, relevant answer, base your response entirely on it, avoiding any external assumptions. "
    "- If the query is within the context’s scope (e.g., drone programming is covered) but no direct answer is found, acknowledge the relevance and suggest a next step (e.g., checking documentation or contacting support) in the query’s language: "
    "  - English: 'This is covered in the database, but I don’t have a direct solution. I suggest checking the relevant documentation or contacting support.' "
    "  - French: 'Ce sujet est couvert dans la base de données, mais je n’ai pas de solution directe. Je suggère de consulter la documentation correspondante ou de contacter le support.' "
    "- If no relevant context is found, respond only with a brief, empathetic message in the query’s language: "
    "  - English: 'Sorry, I don’t have any information on that issue.' "
    "  - French: 'Désolé, je n’ai aucune information sur ce problème.' "
    "- Always end IT-related responses with the fallback message in the query’s language: "
    "  - English: 'If the issue persists, please contact the technical team at +21698252252 or via email: support@tech.example.com.' "
    "  - French: 'Si le problème persiste, contacter les techniciens au +21698252252 ou par email : support@tech.example.com.' "
    "For queries completely unrelated to IT or outside the provided context (e.g., non-technical topics): "
    "- Respond only with a short, empathetic message in the query’s language, without mentioning IT, technical support, or any additional information: "
    "  - English: 'Sorry, that’s not my area, so I can’t help with that.' "
    "  - French: 'Désolé, ce n’est pas mon domaine, je ne peux pas aider.' "
    "Keep responses short, warm, and conversational, like chatting with a friend. "
    "Never invent solutions, provide troubleshooting steps without context, mention technical support for non-IT queries, or include messages in the other language. "
    "Stick strictly to the provided context and these rules to ensure accuracy and helpfulness."
)

# Step 13: Set Up the Chat Engine with Memory
# Assumes 'index' (the VectorStoreIndex) is already created from your RAG setup
chat_engine = ContextChatEngine.from_defaults(
    retriever=index.as_retriever(similarity_top_k=2),  # Retrieves the top 2 most relevant chunks
    llm=llm,  # Uses Mixtral 8x7B via Ollama
    system_prompt=custom_prompt,  # Applies the custom prompt for smart behavior
    chat_history=[]  # Initializes an empty chat history
)
# The chat engine is now ready for you to test with your own queries.

In [14]:
# Test queries
queries = [
    "How do I plan a trip to the moon?",
    "What should I do if my laptop won’t turn on?",
    "Why is my internet connection so slow?"
]
for query in queries:
    response = chat_engine.chat(query)
    print(f"User: {query}")
    print(f"Chatbot: {response.response}")
    print("***********************************************************************")

User: How do I plan a trip to the moon?
Chatbot:  I'm sorry, but that's not something covered in our current context. Space travel isn't typically handled within IT environments. For space-related queries, I would recommend checking resources like NASA or SpaceX for accurate information and guidance on planning a trip to the Moon. If you have any other technical questions, feel free to ask! If the issue persists, please contact the technical team at +21698252252 or via email: support@tech.example.com.
***********************************************************************
User: What should I do if my laptop won’t turn on?
Chatbot:  I'm sorry about that! It seems like your laptop isn't powering up, which can be quite frustrating. In our IT context, this issue is typically handled, so let me see if the provided context can help us.

According to the context information, it looks like you have a DisplayPort problem on a different device (not a laptop). However, I don't see any information